<a href="https://colab.research.google.com/github/quamranhasan/KAIST_CS372-NLP_with_python/blob/main/Assignment3_CS372.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/google-research-datasets/gap-coreference

Cloning into 'gap-coreference'...
remote: Enumerating objects: 19, done.
remote: Total 19 (delta 0), reused 0 (delta 0), pack-reused 19
Unpacking objects: 100% (19/19), 970.66 KiB | 4.60 MiB/s, done.


Read the data file by the following code.

In [ ]:
from collections import defaultdict
import csv
from enum import Enum
import pandas as pd

Develop your program that resolves ambiguous pronouns that refer to names.

In [ ]:
import nltk
import spacy
import pandas as pd

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nlp = spacy.load("en_core_web_sm")


'''
insert three spaces at each offset position in the string and return the modified string
the spaces are placed in front of the words we want to analyze. (name1, name2, ambiguous pronoun)
'''
def mark_string(string, offset_list):
    for offset in offset_list:
        string = string[:offset] + '   ' + string[offset:]
    return string


'''
Function for switching to page-context.
searches if the given names are in the url
if there are spaces replace those with underscores.
'''
def name_is_in_url(name_1, name_2, url):
    mod_name_1 = name_1.replace(' ', '_')
    mod_name_2 = name_2.replace(' ', '_')

    if name_1 in url:
        return 'a'
    elif mod_name_1 in url and ' ' in name_1:
        return 'a'
    elif name_2 in url:
        return 'b'
    elif mod_name_2 in url and ' ' in name_2:
        return 'b'


'''
tokenize mark_string with nlp
identify pronoun from name_list and its dependency relationship in the tokenized string
compare the dependency and determine if find_parallelism exists
return none if no find_parallelism
'''
def find_parallelism(string, offs_list, name_list):
    string_used = mark_string(string, offs_list)
    doc = nlp(string_used)
    pronoun = name_list[2]
    pronoun_dep = ''
    a, b = '', ''

    for token in doc:
        if token.text == pronoun or token.text in pronoun:
            pronoun_dep = token.dep_
            break

    for token in doc:
        if token.text == name_list[0] or token.text in name_list[0]:
            a += token.dep_
        elif token.text == name_list[1] or token.text in name_list[1]:
            b += token.dep_

    if a == pronoun_dep:
        return 'a'
    elif b == pronoun_dep:
        return 'b'


'''
extract features from data
return a dictionary with extracted features

'''
def get_features(row):
    selected_columns = ['A-offset', 'Pronoun-offset', 'B-offset']
    column_lists = []
    for column in selected_columns:
        column_list = row[column]
        column_lists.append(column_list)


    selected_columns2 = ['A', 'B', 'Pronoun']
    column_lists2 = []
    for column in selected_columns2:
        column_list2 = row[column]
        column_lists2.append(column_list2)

    return {
        #comment first one if you want to try snippet context
        #keep both if you want to try page-context
        'name_is_in_url': name_is_in_url(row['A'], row['B'], row['URL']),
        'find_parallelism': find_parallelism(row['Text'], column_lists,column_lists2)
    }

train = pd.read_csv('/content/gap-coreference/gap-development.tsv', sep='\t')
test = pd.read_csv('/content/gap-coreference/gap-test.tsv', sep='\t')

train_feature = [(get_features(row), row['A-coref']) for _, row in train.iterrows()]


#naive bayes classifier that we learnt in class
classifier = nltk.NaiveBayesClassifier.train(train_feature)
print(classifier.show_most_informative_features())

final = []
for _, row in test.iterrows():
    classify = str(classifier.classify(get_features(row)))
    if classify == "True":
        final.append([row['ID'], 'TRUE', 'FALSE'])
    else:
        final.append([row['ID'], 'FALSE', 'TRUE'])


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


Most Informative Features
        find_parallelism = 'b'             False : True   =      5.0 : 1.0
          name_is_in_url = 'a'              True : False  =      4.5 : 1.0
        find_parallelism = 'a'              True : False  =      3.3 : 1.0
          name_is_in_url = 'b'             False : True   =      3.3 : 1.0
          name_is_in_url = None            False : True   =      1.3 : 1.0
        find_parallelism = None            False : True   =      1.0 : 1.0
None


Save the results into a tsv file. The tsv file should have the test ID, True or False for A-coref and B-coref.

In [ ]:
# Example code. You may fix the code to save your answers.
'''writing to tsv'''
with open('result.tsv', 'w', newline = '') as out_file:
    writer = csv.writer(out_file, delimiter='\t')
    for i in final:
        writer.writerow(i)

Code for evaluation. Do not modify this.

In [ ]:
#Do not modify this part
class Gender(Enum):
  UNKNOWN = 0
  MASCULINE = 1
  FEMININE = 2


# Mapping of (lowercased) pronoun form to gender value. Note that reflexives
# are not included in GAP, so do not appear here.
PRONOUNS = {
    'she': Gender.FEMININE,
    'her': Gender.FEMININE,
    'hers': Gender.FEMININE,
    'he': Gender.MASCULINE,
    'his': Gender.MASCULINE,
    'him': Gender.MASCULINE,
}

# Fieldnames used in the gold dataset .tsv file.
GOLD_FIELDNAMES = [
    'ID', 'Text', 'Pronoun', 'Pronoun-offset', 'A', 'A-offset', 'A-coref', 'B',
    'B-offset', 'B-coref', 'URL'
]

# Fieldnames expected in system output .tsv files.
SYSTEM_FIELDNAMES = ['ID', 'A-coref', 'B-coref']

class Annotation(object):
  """Container class for storing annotations of an example.
  Attributes:
    gender(None): The gender of the annotation. None indicates that gender was
      not determined for the given example.
    name_a_coref(None): bool reflecting whether Name A was recorded as
      coreferential with the target pronoun for this example. None indicates
      that no annotation was found for the given example.
    name_b_coref(None): bool reflecting whether Name B was recorded as
      coreferential with the target pronoun for this example. None indicates
      that no annotation was found for the given example.
  """

  def __init__(self):
    self.gender = None
    self.name_a_coref = None
    self.name_b_coref = None


class Scores(object):
  """Container class for storing scores, and generating evaluation metrics.
  Attributes:
    true_positives: Tally of true positives seen.
    false_positives: Tally of false positives seen.
    true_negatives: Tally of true negatives seen.
    false_negatives: Tally of false negatives seen.
  """

  def __init__(self):
    self.true_positives = 0
    self.false_positives = 0
    self.true_negatives = 0
    self.false_negatives = 0

  def recall(self):
    """Calculates recall based on the observed scores.
    Returns:
      float, the recall.
    """
    numerator = self.true_positives
    denominator = self.true_positives + self.false_negatives
    return 100.0 * numerator / denominator if denominator else 0.0

  def precision(self):
    """Calculates precision based on the observed scores.
    Returns:
      float, the precision.
    """
    numerator = self.true_positives
    denominator = self.true_positives + self.false_positives
    return 100.0 * numerator / denominator if denominator else 0.0

  def f1(self):
    """Calculates F1 based on the observed scores.
    Returns:
      float, the F1 score.
    """
    recall = self.recall()
    precision = self.precision()

    numerator = 2 * precision * recall
    denominator = precision + recall
    return numerator / denominator if denominator else 0.0


def read_annotations(filename, is_gold):
  """Reads coreference annotations for the examples in the given file.
  Args:
    filename: Path to .tsv file to read.
    is_gold: Whether or not we are reading the gold annotations.
  Returns:
    A dict mapping example ID strings to their Annotation representation. If
    reading gold, 'Pronoun' field is used to determine gender.
  """

  def is_true(value):
    if value.lower() == 'true':
      return True
    elif value.lower() == 'false':
      return False
    else:
      print('Unexpected label!', value)
      return None

  fieldnames = GOLD_FIELDNAMES if is_gold else SYSTEM_FIELDNAMES

  annotations = defaultdict(Annotation)
  with open(filename, 'rU') as f:
    reader = csv.DictReader(f, fieldnames=fieldnames, delimiter='\t')

    # Skip the header line in the gold data
    if is_gold:
      next(reader, None)

    for row in reader:
      example_id = row['ID']
      if example_id in annotations:
        print('Multiple annotations for', example_id)
        continue

      annotations[example_id].name_a_coref = is_true(row['A-coref'])
      annotations[example_id].name_b_coref = is_true(row['B-coref'])
      if is_gold:
        gender = PRONOUNS.get(row['Pronoun'].lower(), Gender.UNKNOWN)
        assert gender != Gender.UNKNOWN, row
        annotations[example_id].gender = gender
  return annotations


def calculate_scores(gold_annotations, system_annotations):
  """Score the system annotations against gold.
  Args:
    gold_annotations: dict from example ID to its gold Annotation.
    system_annotations: dict from example ID to its system Annotation.
  Returns:
    A dict from gender to a Scores object for that gender. None is used to
      denote no specific gender, i.e. overall scores.
  """
  scores = {}
  for example_id, gold_annotation in gold_annotations.items():
    system_annotation = system_annotations[example_id]

    name_a_annotations = [
        gold_annotation.name_a_coref, system_annotation.name_a_coref
    ]
    name_b_annotations = [
        gold_annotation.name_b_coref, system_annotation.name_b_coref
    ]
    for gender in [None, gold_annotation.gender]:
      if gender not in scores:
        scores[gender] = Scores()

      for (gold, system) in [name_a_annotations, name_b_annotations]:
        if system is None:
          print('Missing output for', example_id)
          scores[gender].false_negatives += 1
        elif gold and system:
          scores[gender].true_positives += 1
        elif not gold and system:
          scores[gender].false_positives += 1
        elif not gold and not system:
          scores[gender].true_negatives += 1
        elif gold and not system:
          scores[gender].false_negatives += 1
  return scores


def make_scorecard(scores):
  """Returns a human-readable scorecard of the given scores.
  Args:
    scores: dict from gender to its Scores object. None is used to denote no
      specific gender, i.e. overall scores.
  Returns:
    A string, the scorecard.
  """
  scorecard = []

  display_names = [(None, 'Overall'), (Gender.MASCULINE, 'Masculine'),
                   (Gender.FEMININE, 'Feminine')]

  bias_terms = {}
  for gender, display_name in display_names:
    gender_scores = scores.get(gender, Scores())

    recall = gender_scores.recall()
    precision = gender_scores.precision()
    f1 = gender_scores.f1()
    bias_terms[gender] = f1

    scorecard.append('{} recall: {:.1f} precision: {:.1f} f1: {:.1f}'.format(
        display_name, recall, precision, f1))
    scorecard.append('\t\ttp {:d}\tfp {:d}'.format(
        gender_scores.true_positives, gender_scores.false_positives))
    scorecard.append('\t\tfn {:d}\ttn {:d}'.format(
        gender_scores.false_negatives, gender_scores.true_negatives))

  bias = '-'
  if bias_terms[Gender.MASCULINE] and bias_terms[Gender.FEMININE]:
    bias = '{:.2f}'.format(
        bias_terms[Gender.FEMININE] / bias_terms[Gender.MASCULINE])
  scorecard.append('Bias (F/M): {}\n'.format(bias))
  return '\n'.join(scorecard)


def run_scorer(gold_tsv, system_tsv):
  """Run the scorer.
  Args:
    gold_tsv: Gold annotations to score against.
    system_tsv: System output to score.
  Returns:
    A string, the scorecard.
  """
  gold_annotations = read_annotations(gold_tsv, is_gold=True)
  assert gold_annotations, 'No gold annotations read!'

  system_annotations = read_annotations(system_tsv, is_gold=False)
  assert system_annotations, 'No system annotations read!'

  scores = calculate_scores(gold_annotations, system_annotations)
  return make_scorecard(scores)

In [ ]:
gold_tsv = '/content/gap-coreference/gap-test.tsv'
system_tsv = '/content/result.tsv'

Set the file paths.

gold_tsv = the test data

system_tsv = your answer

Check your results here. You may modify this part if you want to.

In [ ]:
scorecard = run_scorer(gold_tsv, system_tsv)
print(scorecard)

Overall recall: 69.3 precision: 61.4 f1: 65.1
		tp 1228	fp 772
		fn 545	tn 1455
Masculine recall: 69.6 precision: 61.9 f1: 65.5
		tp 619	fp 381
		fn 270	tn 730
Feminine recall: 68.9 precision: 60.9 f1: 64.6
		tp 609	fp 391
		fn 275	tn 725
Bias (F/M): 0.99



<ipython-input-5-330a7177ddfe>:115: DeprecationWarning: 'U' mode is deprecated
  with open(filename, 'rU') as f:
